# Client

In [6]:
import socket , cv2 , numpy
import threading


In [7]:
def send():
    while True:
        ret,video = cam.read()
        photo = cv2.resize(video, (640, 480))
        s.send(numpy.ndarray.tobytes(video))
        #s.send(input("\n ").encode())


In [8]:
def recv():
    while True:
        data = s.recv(100000000)
        photo =  numpy.frombuffer(data, dtype=numpy.uint8)
        if len(photo) == 640*480*3:
            cv2.imshow('From Server', photo.reshape(480, 640, 3))
            if cv2.waitKey(10) == 13:
                cv2.destroyAllWindows()
                cam.release()
                break
        #msg=s.recv(1024)
        #print("\n\t\t\t\t\tReceived:- {}".format(addr,msg.decode()))


In [9]:

print("\t\t\t\t\t........WELCOME TO MY LIVE CHATE........\n\n")
ip= "192.168.43.192"
port= 1234
 
s = socket.socket(socket.AF_INET , socket.SOCK_STREAM)
s.connect(( ip , int(port)))
cam = cv2.VideoCapture(0)


t1=threading.Thread(target = send)
t2=threading.Thread(target = recv)


t2.start()
t1.start()

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\lenovo\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\lenovo\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\lenovo\anaconda3\lib\threading.py", line 870, in run
    self.run()
  File "C:\Users\lenovo\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-b3aaaabca22c>", line 5, in send
    self._target(*self._args, **self._kwargs)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
  File "<ipython-input-3-3be2620fff16>", line 3, in recv
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host


					........WELCOME TO MY LIVE CHATE........




In [5]:
cam.release()


In [ ]:
#importing library

import socket
import numpy as np
import cv2 as cv
import threading

# client program socket to connect to the server program
skt = socket.socket()
skt.bind(("", 4321))  # empty means local system
server_ip = input("enter server ip: ")
server_port = input("enter server port number: ")

skt.connect((server_ip, int(server_port))) 
skt.send(b"connected")  # sending string as data
cameraIndex = 0 # the camera to use i.e. laptop webcam
camera = cv.VideoCapture(cameraIndex) # starting the camera

# function for cleint to work as receiver
def receiver():
    framesLost = 0
    print("Entered")
    while True:
        framesLost += 1 # counting frame
        data = skt.recv(100000000)  # receiving data with the size limit
        if(data == b'finished'): # to stop receiving and stop camera
            print("Finished")
            camera.release()
            skt.close()
        else:  # converting the byte data into numpy array
            photo =  np.frombuffer(data, dtype=np.uint8)
            if len(photo) == 640*480*3: # changing the array shape and getting the video
                cv.imshow('From Server', photo.reshape(480, 640, 3))
                if cv.waitKey(1) == 13: # camera closing condition
                    skt.send(b'finished')
                    camera.release()
                    cv.destroyAllWindows()
                    break
            else:
                print("Lost {} frames".format(framesLost) ) #counting the lost frames during video exchange

# function for client to send the data of the video             
def sender():
    while True: # reading the camera data resizing and sending it as byte 
        status, photo = camera.read()
        photo = cv.resize(photo, (640, 480))
        #photo = cv.resize(photo, (600, 500))
        print(photo.shape)
        if status:
            skt.send(np.ndarray.tobytes(photo))
        else: print("Could not get frame")
    camera.release()

# threads to run both the functions
threading.Thread(target=receiver).start()
threading.Thread(target=sender).start()